In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import re
from sklearn.linear_model import LinearRegression
from fancyimpute import IterativeImputer
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import KNNImputer
from sklearn.metrics import confusion_matrix, accuracy_score
import appmode
from keras.models import model_from_json

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

### Import Dataset
Let's import the dataset we saved from the C3.ai API

In [3]:
dataset = pd.read_csv('./files/Covid19_LineList_Records.csv')
pd.set_option('display.max_columns', None)
dataset.head()

,Unnamed: 0,age,gender,caseConfirmationDate,livesInGroundZero,traveledToGroundZero,recordSource,lineListSource,hasTravelHistory,id,location.id,locationType,groundZeroExposure,notes,relevantTravelHistoryLocation,sequenceAvailable,symptomStartDate,hospitalAdmissionDate,relevantTravelHistoryDates,outcome,didDie,didRecover,outcomeDate,symptoms,ageRange,chronicDisease
0,0,27.0,female,2020-04-04T00:00:00Z,False,False,https://www.prensalibre.com/guatemala/comunita...,OPEN,False,00002e80-d790-4d77-aa02-7da86338c41b,GuatemalaCity_GuatemalaCity_Guatemala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,2020-03-26T00:00:00Z,False,False,NaN,OPEN,False,00004cec-61fe-43a0-b4c4-cbb2a2c4e932,NewcastleuponTyneTyneandWear_England_UnitedKin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,2020-03-25T00:00:00Z,False,False,https://coronavirus.health.ny.gov/county-count...,OPEN,False,00009339-ab52-4de7-afad-3a8ca35f700e,NewYork_UnitedStates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,2020-03-22T00:00:00Z,False,False,https://coronavirus.health.ny.gov/county-count...,OPEN,False,0000ef82-6365-4ee6-bb44-00b8575dbf3c,NewYorkCity_NewYork_UnitedStates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,2020-03-29T00:00:00Z,False,False,https://www.mercurynews.com/2020/03/20/map-cor...,OPEN,False,000174f4-8b80-4832-b7d5-333e4a3ccfcf,California_UnitedStates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#GET DATASET INFO
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266871 entries, 0 to 266870
Data columns (total 26 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     266871 non-null  int64  
 1   age                            11959 non-null   float64
 2   gender                         15510 non-null   object 
 3   caseConfirmationDate           263353 non-null  object 
 4   livesInGroundZero              266871 non-null  bool   
 5   traveledToGroundZero           266871 non-null  bool   
 6   recordSource                   247324 non-null  object 
 7   lineListSource                 266871 non-null  object 
 8   hasTravelHistory               266871 non-null  bool   
 9   id                             266871 non-null  object 
 10  location.id                    266848 non-null  object 
 11  locationType                   6868 non-null    object 
 12  groundZeroExposure            

### Clean The Dataset
We can determine that age, gender, symptoms, location and chronic disease will be the most useful features in predicting whether a patient did or will die or not.



In [5]:
#Create new df out of only the columns we want
patient_level = dataset[["age", "gender", "symptoms", "location.id", "chronicDisease", "didDie"]]
patient_level.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266871 entries, 0 to 266870
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   age             11959 non-null   float64
 1   gender          15510 non-null   object 
 2   symptoms        1740 non-null    object 
 3   location.id     266848 non-null  object 
 4   chronicDisease  210 non-null     object 
 5   didDie          1004 non-null    object 
dtypes: float64(1), object(5)
memory usage: 12.2+ MB


### Note: Limitations
Looking at the original dataset, we see there are just over a quarter of a million records. However, well over 50% of records are missing data in columns that are crucial to our model, particularly in the column that holds values we are trying to predict (didDie). <br><br>This goes beyond  the threshold of what I'd find to be an acceptable amount of missing data, as attempting to impute 200k+ records, using only a fraction of that, would likely introduce bias, altered variance, and an overall skewed dataset.<br>


In [6]:
#Drop null values
patient_level = patient_level.dropna(subset = ["didDie"])

patient_level

,age,gender,symptoms,location.id,chronicDisease,didDie
183,43.0,male,NaN,XiningCity_Qinghai_China,NaN,False
875,48.0,female,NaN,HougangStreet61__Singapore,NaN,False
925,58.0,female,NaN,NationalCentreforInfectiousDiseases__Singapore,NaN,False
931,NaN,NaN,NaN,Boone_Missouri_UnitedStates,NaN,True
983,NaN,NaN,NaN,Huai'anCity_Jiangsu_China,NaN,False
...,...,...,...,...,...,...
265358,65.0,male,acute respiratory distress syndrome:pneumonia,Quezon_MetroManila_Philippines,hypertension:chronic kidney disease,True
265423,NaN,NaN,NaN,YanchengCity_Jiangsu_China,NaN,False
265720,57.0,male,cardiogenic shock:acute coronary syndrome:hear...,Manila_MetroManila_Philippines,hypertension,True
265851,NaN,NaN,NaN,Huai'anCity_Jiangsu_China,NaN,False


Here we can see our new working dataset is much smaller, with only 642 records. Tho this is significantly refined from the 250k+ records from the original dataset, it is much more managable for dealing with the proportionately acceptable amount of null values, while still being just enough records to create a valid machine learning model, and create observational insights.<br><br>
We will use a KNN imputer method to fill in the missing values in the "age" column. While there is missing data in the "chronicDisease" column, due to the nature of the data (detailing whether or not a certain patient had pre-existing conditions), we can assume that a null value either means one of two things: 
- The patient had no underlying conditions
- The value is null/missing whether the patient had underlying conditions or not

I personally decided not to try to impute this data because it first of all, very difficult to obtain on a per record basis. Secondly, all but one record had the value "none" in the chronic disease column (indicating no pre-existing condition). Using something like KNN, or a regression to try to impute null values in this column would likely result in biased data, as the imputation would presumably try to fill _each_ null field with a chronic disease, instead of recognizing that some patients may have "none" as a value.

In [7]:
#VISUALIZE NEW DATASET
patient_level.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1004 entries, 183 to 265871
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             577 non-null    float64
 1   gender          642 non-null    object 
 2   symptoms        301 non-null    object 
 3   location.id     1004 non-null   object 
 4   chronicDisease  123 non-null    object 
 5   didDie          1004 non-null   object 
dtypes: float64(1), object(5)
memory usage: 54.9+ KB


In [8]:
#COUNT NULLS
patient_level.isna().sum()

age               427
gender            362
symptoms          703
location.id         0
chronicDisease    881
didDie              0
dtype: int64

### NOTE:
We notice that some values in the "symptoms" and "chronicDisease" columns are comma or colon separated strings. Some strings are also mispelled, and/or duplicates with slightly different wording. We'll have to fix this so we don't create unnecessary and redundant features that will skew our model. I aslo decide to remove "location.id" from our final dataset, as I don't find it to be a useful predictive feature in this case. <br><br>Looking at the value counts for "location.id"(country), most records are largely from 3 eastern Asian countries. There are a host of social, political or demographic attributes that could influence health outcomes for people from a particular nation. For that reason, and since we are not only looking at making predictions for one nation, I decided to remove the country feature, so as to not introduce bias to the model.

In [9]:
#CHOOSE COLUMNS TO MAKE INTO NEW DF
patient_level2 = dataset[["age", "gender", "symptoms", "location.id", "chronicDisease", "didDie"]]

#DROP NULL VALUES
patient_level2 = patient_level2.dropna(subset = ["didDie", "gender"])

#REPLACE NAN VALUES IN "SYMPTOMS" and "CHRONIC DISEASE" COLUMN WITH "NONE"
patient_level2['symptoms'] = patient_level2['symptoms'].fillna('asymptomatic')
patient_level2['chronicDisease'] = patient_level2['chronicDisease'].fillna('none')

#STRIP ALL CHARACTERS BEFORE THE LAST "_" IN THE LOCATION.ID COLUMN TO RETURN JUST THE NAME OF THE COUNTRY
patient_level2['location.id'] = patient_level2['location.id'].str.split('_').str[-1].str.strip()

#REPLACE THE COLONS AND SEMICOLONS WITH COMMAS, SPLIT VALUES BY COMMA
patient_level2['symptoms'] = patient_level2['symptoms'].str.replace(':', ',')
patient_level2['symptoms'] = patient_level2['symptoms'].str.replace(';', ',')
patient_level2['symptoms'] = patient_level2['symptoms'].str.split(",")

patient_level2['chronicDisease'] = patient_level2['chronicDisease'].str.replace(':', ',')
patient_level2['chronicDisease'] = patient_level2['chronicDisease'].str.replace(';', ',')
patient_level2['chronicDisease'] = patient_level2['chronicDisease'].str.split(",")

#CLEAN UP SYMPTOMS THAT MEAN THE SAME THING BUT ARE SPELLED OR WORDED DIFFERENTLY
patient_level2['symptoms'] = [['acute respiratory distress' if 'respiratory' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['acute kidney injury' if 'acute renal failure' in s else 
                               'acute kidney injury' if 'acute kidney injury' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['fatigue' if 'fatigue' in s else 'fatigue' if 'somnolence' in s else 'fatigue' if 'fatigure' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['shortness of breath' if ' breathing' in s else 'shortness of breath' if 'dyspnea' 
                               in s else 'shortness of breath' if 'shortness of breath' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['cold chills' if 'chill' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['acute respiratory distress' if 'distress' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['pneumonia' if 'pneumonia' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['septic shock' if 'sepsis' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['arrhythmia' if 'arrhythmia' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['asymptomatic' if 'none' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['cough' if 'cough' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['runny nose' if 'running nose' in s else 'runny nose' if 'runny nose' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['malaise' if 'malaise' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['muscular soreness' if 'myalgias' in s else 'muscular soreness' if 'mialgia' in s else 'muscular soreness' if 
                               'myalgia' in s else 'muscular soreness' if 'muscular soreness' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['fever' if 'fever' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['sore throat' if 'sore throat' in s else s for s in lst] for lst in patient_level2['symptoms']]
patient_level2['symptoms'] = [['headache' if 'headache' in s else s for s in lst] for lst in patient_level2['symptoms']]

#CLEAN UP CHRONIC DISEASES THAT MEAN THE SAME THING BUT ARE SPELLED OR WORDED DIFFERENTLY
patient_level2['chronicDisease'] = [['hypertension' if 'Hypertension' in s else 'hypertension' if 'hypertenstion' in s else
                                     'hypertension' if 'hypertension' in s else s for s in lst] for lst in patient_level2['chronicDisease']]
patient_level2['chronicDisease'] = [['diabetes' if 'diabetes' in s else 'diabetes' if 'Diabetes' in s else s for s in lst] for lst in patient_level2['chronicDisease']]
patient_level2['chronicDisease'] = [['COPD' if 'chronic obstructive pulmonary disease' in s else s for s in lst] for lst in patient_level2['chronicDisease']]
patient_level2['chronicDisease'] = [['asthma' if 'asthma' in s else s for s in lst] for lst in patient_level2['chronicDisease']]
patient_level2['chronicDisease'] = [['coronary heart disease' if 'coronary' in s else s for s in lst] for lst in patient_level2['chronicDisease']]
patient_level2['chronicDisease'] = [['kidney disease' if 'kidney' in s else 'kidney disease' if 'renal' in s else s for s in lst] for lst in patient_level2['chronicDisease']]
patient_level2['chronicDisease'] = [['cancer' if 'cancer' in s else s for s in lst] for lst in patient_level2['chronicDisease']]
patient_level2['chronicDisease'] = [['parkinson\'s disease' if 'Parkinson\'s disease' in s else s for s in lst] for lst in patient_level2['chronicDisease']]
patient_level2['chronicDisease'] = [['heart disease' if 'cardi' in s else 'heart disease' if 'heart' in s else 
                                     'heart disease' if 'atrial' in s else 'heart disease' if 'vent' in s else s for s in lst] for lst in patient_level2['chronicDisease']]
patient_level2['chronicDisease'] = [['cerebrovascular disease' if 'cereb' in s else 
                                     'cerebrovascular disease' if 'encephalomalacia' in s else s for s in lst] for lst in patient_level2['chronicDisease']]
patient_level2['chronicDisease'] = [['thyroid disease' if 'thyroid' in s else s for s in lst] for lst in patient_level2['chronicDisease']]
patient_level2['chronicDisease'] = [['enlarged prostate' if 'prostat' in s else s for s in lst] for lst in patient_level2['chronicDisease']]
patient_level2['chronicDisease'] = [['dyslipidemia' if 'dislipidemia' in s else s for s in lst] for lst in patient_level2['chronicDisease']]

#RENAME LOCATION.ID COLUMN
patient_level2 = patient_level2.rename(columns={'location.id': 'country'})


#patient_level2.columns = patient_level2.columns.str.strip()
#patient_level2 = pd.concat([patient_level2.drop('symptoms', 1), patient_level2['symptoms'].str.get_dummies(sep=",")], 1)


patient_level2

,age,gender,symptoms,country,chronicDisease,didDie
183,43.0,male,[asymptomatic],China,[none],False
875,48.0,female,[asymptomatic],Singapore,[none],False
925,58.0,female,[asymptomatic],Singapore,[none],False
996,46.0,male,[fever],China,[none],False
1012,70.0,male,[asymptomatic],Japan,[none],False
...,...,...,...,...,...,...
264532,70.0,male,[Severe],Gambia,[diabetes],False
264567,47.0,female,[asymptomatic],Singapore,[none],False
265358,65.0,male,"[acute respiratory distress, pneumonia]",Philippines,"[hypertension, kidney disease]",True
265720,57.0,male,"[cardiogenic shock, acute coronary syndrome, h...",Philippines,[hypertension],True


### NOTE: 
there are multiple symptoms and chronic disease per patient (in some cases) represented as a list in the repsective column, we'll have to take each value out of list form, and tranpose them to their own columns, making them dummy variables with a binary value. This way our model can differentiate between someone having multiple symptoms/conditons, or just one. <br><br>If we don't do this, a categorical encoder will recognize someone having 2 symptoms for instance (cough and fever), as one category, while a record reporting just 1 symptom (cough) would be a separate category, and create innaccuracies in our model.<br>


In [10]:
#CREATE DUMMY VARIABLES FOR EACH VALUE IN THE LISTS THAT MAKE UP THE SYMPTOMS COLUMN
symptomDummies = pd.concat([patient_level2.drop('symptoms', 1), patient_level2['symptoms'].str.join('|').str.get_dummies()], 1)

#STRIP LEADING SPACES FROM COLUMN NEW NAMES
symptomDummies.columns = symptomDummies.columns.str.strip()

#REMOVE SPECIFIC DUMMY COLUMNS WE DON'T WANT (WE'RE ONLY KEEPING THE MOST COMMON SYMPTOMS)
symptomDummies = symptomDummies.drop(['severe', 'gasp', 'grasp', 'conjunctivitis', 'weak', 'afebrile', 'Severe', 'dizziness', 'multiple electrolyte imbalance', 
                                      'multiple organ failure', 'myocardial dysfunction', 'primary myelofibrosis', 'chest discomfort', 'cardiopulmonary arrest',
                                     'dysphagia', 'discomfort', 'sputum', 'little sputum', 'expectoration', 'systemic weakness', 'anorexia', 'lesions on chest radiographs',
                                     'cardiogenic shock', 'kidney failure and hypertension', 'obnubilation', 'gastritis', 'eye irritation',
                                     'emesis', 'myocardial infarction', 'acute coronary syndrome', 'colds', 'hypoxia', 'discomfort', 
                                      'heart failure', 'acute kidney injury', 'arrhythmia', 'acute myocardial infarction', 'chest pain', 'congestive heart failure'], axis=1)


#new = symptomDummies.loc[symptomDummies['septic shock']==1]
#new

symptom_cd_dummies = pd.concat([symptomDummies.drop('chronicDisease', 1), symptomDummies['chronicDisease'].str.join('|').str.get_dummies()], 1)

#STRIP LEADING SPACES FROM COLUMN NEW NAMES
symptom_cd_dummies.columns = symptom_cd_dummies.columns.str.strip()

symptom_cd_dummies = symptom_cd_dummies.drop(['"thought to have had other pre-existing conditions"', 'hemorrhage of digestive tract', 
                                              'upper git bleeding', 'taking medicine of Madopar', 'hip replacement', 'hypertensive', 'impaired fasting glucose', 'country'], axis=1)

#MOVE didDie COLUMN TO THE END
move_column = symptom_cd_dummies.pop("didDie")
symptom_cd_dummies.insert(34, move_column.name, move_column)

#CHANGE VALUES IN THE didDie COLUMN TO BINARY
die = {True: 1, False: 0}
symptom_cd_dummies.didDie = [die[item] for item in symptom_cd_dummies.didDie]

#symptom_cd_dummies.to_csv("symptom_cd_dummies.csv")
symptom_cd_dummies

,age,gender,acute respiratory distress,asymptomatic,cold chills,cough,diarrhea,fatigue,fever,headache,malaise,muscular soreness,pneumonia,runny nose,septic shock,shortness of breath,sore throat,Tuberculosis,COPD,asthma,atherosclerosis,cancer,cerebrovascular disease,chronic bronchitis,diabetes,dyslipidemia,enlarged prostate,heart disease,hepatitis B,hypertension,kidney disease,none,parkinson's disease,thyroid disease,didDie
183,43.0,male,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
875,48.0,female,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
925,58.0,female,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
996,46.0,male,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1012,70.0,male,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264532,70.0,male,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
264567,47.0,female,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
265358,65.0,male,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1
265720,57.0,male,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


### NOTE:
We can use this to get an idea of the most common symptoms and chronic disease to help us decide which are worth keeping in our model as features

In [11]:
#GET A COUNT OF VALUES FOR SYMPTOMS AND CHRONIC DISEASES
counts = symptom_cd_dummies.drop(columns=['age', 'gender'], axis=0)
counts = counts.melt(var_name='columns', value_name='index')
counts = pd.crosstab(index=counts['index'], columns=counts['columns'])
counts.sort_values(by=1, axis=1, ascending=False, inplace=True)
counts

columns,none,asymptomatic,didDie,fever,hypertension,pneumonia,cough,acute respiratory distress,diabetes,septic shock,kidney disease,heart disease,fatigue,sore throat,shortness of breath,runny nose,headache,muscular soreness,asthma,COPD,cold chills,cancer,enlarged prostate,malaise,cerebrovascular disease,dyslipidemia,diarrhea,chronic bronchitis,thyroid disease,hepatitis B,parkinson's disease,atherosclerosis,Tuberculosis
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,120,209,454,551,561,575,577,579,584,623,624,624,626,627,630,632,633,634,634,635,635,636,638,638,638,639,639,639,639,640,640,640,641
1,522,433,188,91,81,67,65,63,58,19,18,18,16,15,12,10,9,8,8,7,7,6,4,4,4,3,3,3,3,2,2,2,1


In [12]:
#CHANGE VALUES IN THE gender COLUMN TO BINARY SO IT CAN BE USED IN LINEAR REG TO PREDICT MISSING AGE VALUES
#maleFemale = {'male': 1, 'female': 0}
#symptom_cd_dummies.gender = [maleFemale[item] for item in symptom_cd_dummies.gender]

### Data Preprocessing for machine learning model

In [13]:
x = symptom_cd_dummies.iloc[:, :-1].values
y = symptom_cd_dummies.iloc[:, -1].values

#LABEL ENCODE THE GENDER COLUMN
le = LabelEncoder()
x[:, 1] = le.fit_transform(x[:, 1])
#x[:,0]

In [14]:
#IMPUTE MISSING VALUES IN THE AGE COLUMN
imputer = KNNImputer(n_neighbors=2)

x = imputer.fit_transform(x)

x

array([[43.,  1.,  0., ...,  1.,  0.,  0.],
       [48.,  0.,  0., ...,  1.,  0.,  0.],
       [58.,  0.,  0., ...,  1.,  0.,  0.],
       ...,
       [65.,  1.,  1., ...,  0.,  0.,  0.],
       [57.,  1.,  0., ...,  0.,  0.,  0.],
       [77.,  0.,  0., ...,  1.,  0.,  0.]])

### Split the cleaned dataset into Train and Test sets

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [16]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

### Build the Artificial Neural Network

In [17]:
#Initializing the ANN
ann = tf.keras.models.Sequential()

#ADDING THE SECOND HIDDEN LAYER
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

#ADDING THE OUTPUT LAYER
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Training The ANN

In [18]:
#COMPILING THE ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#TRAINING THE ANN
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
17/17 [==============================] - 0s 2ms/step - loss: 0.8351 - accuracy: 0.6589
Epoch 2/100
17/17 [==============================] - 0s 2ms/step - loss: 0.7620 - accuracy: 0.6959
Epoch 3/100
17/17 [==============================] - 0s 2ms/step - loss: 0.7072 - accuracy: 0.6998
Epoch 4/100
17/17 [==============================] - 0s 3ms/step - loss: 0.6567 - accuracy: 0.7057
Epoch 5/100
17/17 [==============================] - 0s 3ms/step - loss: 0.6198 - accuracy: 0.7154
Epoch 6/100
17/17 [==============================] - 0s 2ms/step - loss: 0.5861 - accuracy: 0.7173
Epoch 7/100
17/17 [==============================] - 0s 2ms/step - loss: 0.5544 - accuracy: 0.7271
Epoch 8/100
17/17 [==============================] - 0s 2ms/step - loss: 0.5231 - accuracy: 0.7544
Epoch 9/100
17/17 [==============================] - 0s 4ms/step - loss: 0.4992 - accuracy: 0.7797
Epoch 10/100
17/17 [==============================] - 0s 3ms/step - loss: 0.4780 - accuracy: 0.7914
Epoch 11/

17/17 [==============================] - 0s 2ms/step - loss: 0.2704 - accuracy: 0.8928
Epoch 84/100
17/17 [==============================] - 0s 3ms/step - loss: 0.2696 - accuracy: 0.8928
Epoch 85/100
17/17 [==============================] - 0s 2ms/step - loss: 0.2687 - accuracy: 0.8928
Epoch 86/100
17/17 [==============================] - 0s 2ms/step - loss: 0.2684 - accuracy: 0.8967
Epoch 87/100
17/17 [==============================] - 0s 2ms/step - loss: 0.2675 - accuracy: 0.8967
Epoch 88/100
17/17 [==============================] - 0s 2ms/step - loss: 0.2669 - accuracy: 0.8967
Epoch 89/100
17/17 [==============================] - 0s 2ms/step - loss: 0.2668 - accuracy: 0.8967
Epoch 90/100
17/17 [==============================] - 0s 2ms/step - loss: 0.2663 - accuracy: 0.8967
Epoch 91/100
17/17 [==============================] - 0s 3ms/step - loss: 0.2656 - accuracy: 0.8967
Epoch 92/100
17/17 [==============================] - 0s 3ms/step - loss: 0.2652 - accuracy: 0.8967
Epoch 93/100


### Saving the model for later use

#SERIALIZE MODEL TO JSON
model_json = ann.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

#SERIALIZE WEIGHTS TO HDF5
ann.save_weights("model.h5")
print("Saved model to disk")

#LOAD JSON AND CREATE MODEL
json_file = open('model.json', 'r')
loaded_model_from_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_from_json)

#LOAD WEIGHTS INTO NEW MODEL
loaded_model.load_weights("model.h5")

#EVALUATE LOADED MODEL ON TEST DATA
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

y_pred = loaded_model.predict(x_test)
y_pred = (y_pred > 0.5)
compare = pd.DataFrame(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
compare.head()

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

### Predicting Test Set Results

In [19]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
compare = pd.DataFrame(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
compare

,0,1
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
124,0,0
125,0,0
126,0,0
127,0,0


### Making Confusion Matrix

In [20]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy = accuracy_score(y_test, y_pred)
accuracy

[[85  1]
 [14 29]]


0.8837209302325582

### Predicting a new single result:
#### Possible features (in order of index): 
    0. age
    1. gender (binary value)
#### Possible symptoms (in order of index, as binary values):
    2. acute respiratory distress   7. fatigue               12. pneumonia 
    3. asymptomatic                 8. fever                 13. runny nose
    4. cold chills                  9. headache              14. septic shock
    5. cough                        10. malaise               15. shortness of breath
    6. diarrhea                     11. muscular soreness    16. sore throat
   
    
#### Possible chronic conditions (in order of index, as binary values):
    17. tuberculosis                 
    18. COPD                         26. enlarged prostate
    19. asthma                       27. heart disease
    20. atherosclerosis              28. hepatitis B
    21. cancer                       29. hypertension
    22. cerebrovascular disease	  30. kidney disease
    23. chronic bronchitis           31. none (not known)
    24. diabetes                     32. parkinson's disease
    25. dyslipidemia                 33. thyroid disease

In [21]:
#PREDICTING THE OUTCOME OF A 65y/o MALE PATIENT WITH COPD
print(ann.predict(sc.transform([[65,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])) > 0.5)
print(ann.predict(sc.transform([[65,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])))

[[False]]
[[0.13276854]]


## Making a program that allows users to make predictions on a per patient basis based on input

In [22]:
from IPython.display import display, Markdown

#INITIALIZE LIST TO HOLD MATRIX OF FEATURES
mylist = list(range(34))

#INPUT PATIENT AGE AND GENDER
display(Markdown("### Demographics:"))
age = int(input("PROMPT 1: What is the patient's age? \n-\n"))
gender = input("\nPROMPT 2: What is the patient's sex? (write 'male' or 'female')\n-\n")
genderNum = 1 if gender=='male' else 0

#PROMPT USER INPUT OF SYMPTOMS
display(Markdown("### Health Profile:"))
display(Markdown("#### Symptoms:"))
print("2. acute respiratory distress   3. asymptomatic   4. cold chills   5. cough\n6. diarrhea   7. fatigue   8. fever.  9.headache.  10. malaise\n11. muscle soreness   12. pneumonia   13. runny nose   14. septic shock   15. shortness of breath\n16. sore throat")

#PROMPT USER INPUT OF PRE-EXISTING CONDITIONS
display(Markdown("#### Pre-existing Conditions:"))
print("17. tuberculosis   18.COPD   19.asthma   20. atherosclerosis   21. Cancer   22. cerebrovascular disease\n23. chronic bronchitis  24. diabetes   25. dyslipidemia   26. enlarged prostate   27. heart disease\n28. Hepatitis B   29. hypertension   30. kidney disease.  31. none   32. parkinson's disease.  33. thyroid disease\n")

symptoms = [int(x) for x in input("\nPROMPT 3: Enter the number next to the patient's corresponding symptoms, and/or pre-existing conditions as listed, \nseparated by a space:\n\n").split()]
mylist2 = [1 if int(el)==int(el) in symptoms else 0 for el in mylist]
  

mylist2[0] = age
mylist2[1] = genderNum

#RETURN THE PROBABILITY PREDICTION
probability = ann.predict(sc.transform([mylist2]))
probabilityPercent = probability.round(2)

#RETURN THE PREDICTION IN FORM OF TRUE/FALSE
hardPrediction = probability>0.5

#RETURN THE 'TRUE/FALSE' VALUE OF 'hardPredction' WITHOUT THE BRACKETS
hardPrediction = ''.join(map(str, hardPrediction[0]))

class color:
   BOLD = '\033[1m'
   RED = '\033[91m'
   GREEN = '\033[92m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def final_prediction(finalResult):
    if (probability > 0.5):
        print(color.BOLD + hardPrediction + color.END)
        print("Our model predicts that a fatal infection is likely for this patient")
        print(color.RED + "HIGH RISK" + color.END)
    else:
        print(color.BOLD + hardPrediction + color.END)
        print("Our model predicts that a fatal infection is not likely for this patient")
        print(color.GREEN + "LOW RISK" + color.END)     

print(color.UNDERLINE + "\nPredicted probability of mortality:" + color.END)
print(*probability[0])
print(color.BOLD + "{0} %\n".format(*probabilityPercent[0]*100) + color.END)
print(color.UNDERLINE + "Fatal infection prediction:" + color.END)
final_prediction(hardPrediction)


### Demographics:

PROMPT 1: What is the patient's age? 
-
34

PROMPT 2: What is the patient's sex? (write 'male' or 'female')
-
male


### Health Profile:

#### Symptoms:

2. acute respiratory distress   3. asymptomatic   4. cold chills   5. cough
6. diarrhea   7. fatigue   8. fever.  9.headache.  10. malaise
11. muscle soreness   12. pneumonia   13. runny nose   14. septic shock   15. shortness of breath
16. sore throat


#### Pre-existing Conditions:

17. tuberculosis   18.COPD   19.asthma   20. atherosclerosis   21. Cancer   22. cerebrovascular disease
23. chronic bronchitis  24. diabetes   25. dyslipidemia   26. enlarged prostate   27. heart disease
28. Hepatitis B   29. hypertension   30. kidney disease.  31. none   32. parkinson's disease.  33. thyroid disease


PROMPT 3: Enter the number next to the patient's corresponding symptoms, and/or pre-existing conditions as listed, 
separated by a space:

2

Predicted probability of mortality:
0.7400686
74.0 %

Fatal infection prediction:
True
Our model predicts that a fatal infection is likely for this patient
HIGH RISK


## Conclusion
Here we've created a Deep Learning predictive model with a {{accuracy}} accuracy score. Of course, thea accuracy of the model could be improved by tweaking the hidden layers and other parameters. The accuracy of its predictions are also accurate relative to the info fed into the model from our very small dataset, which was represntative of a small, particular population of people from mostly eastern asian countries. Theoretically, this model can be applied to larger datasets that are representative of broader/different populations, and perform just as accurately, relative to the populations represented in those datasets. 